# Disabling Autoshape 

The yolov5 model takes an image as input. The AutoShape functionality allows this image to be of any size and type, however because of this it's not possible to convert the end-to-end model with AutoShape into TorchScript.

This notebook takes out the appropriate parts of AutoShape into functions and verifies the behaviour is unchanged.

In [1]:
import sys
sys.path.insert(0,'../street2sat_utils/')

In [2]:
import torch
import numpy as np
import cv2
from yolov5 import hubconf
from yolov5.utils.datasets import letterbox
from yolov5.utils.general import make_divisible, non_max_suppression, scale_coords
from yolov5.models.common import Detections, AutoShape

from matplotlib import pyplot as plt
%matplotlib inline 

In [4]:
img1_path = '../example_images/GP__1312.JPG'
img2_path = '../example_images/GP__1313.JPG'
img3_path = '../example_images/GP__1314.JPG'

img1 = cv2.cvtColor(cv2.imread(img1_path), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(img2_path), cv2.COLOR_BGR2RGB)
img3 = cv2.cvtColor(cv2.imread(img3_path), cv2.COLOR_BGR2RGB)
print(img1.shape, img2.shape, img3.shape)

(2028, 2704, 3) (2028, 2704, 3) (2028, 2704, 3)


## Loading models

In [7]:
model_auto = hubconf.custom("../model_weights/yolo/best.pt", autoshape=True)
model_no = hubconf.custom("../model_weights/yolo/best.pt", autoshape=False)

Fusing layers... 
/Users/izvonkov/nasaharvest/street2sat_website/venv/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 391 layers, 21100857 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 d4c76ee torch 1.9.0 CPU

Fusing layers... 
Model Summary: 391 layers, 21100857 parameters, 0 gradients
YOLOv5 🚀 d4c76ee torch 1.9.0 CPU



## Original Model with AutoShape

In [8]:
result_1 = model_auto(img1).pandas().xyxy[0]
result_1.head()

xmin        ymin         xmax         ymax  confidence  class  \
0   474.721436  834.870789   646.257874  1155.997437    0.654839     11   
1  1242.162109  750.757385  1398.083252  1085.277710    0.628839     11   
2  1986.495850  783.639343  2100.967285  1043.109863    0.604560     11   
3  1842.620483  786.651917  1964.361328  1052.111084    0.593445     11   
4   787.024536  837.682739   903.177734  1124.324341    0.581062     11   

        name  
0  sugarcane  
1  sugarcane  
2  sugarcane  
3  sugarcane  
4  sugarcane

## AutoShape used explicitly

In [9]:
m = AutoShape(model_no)
m.stride = model_no.stride
m.names = model_no.names
result_2 = m(img1).pandas().xyxy[0]

In [10]:
result_1.equals(result_2)

True

## Reproducing Autoshape

In [11]:
# Reproduce preprocessing from AutoShape https://github.com/ultralytics/yolov5/blob/master/models/common.py
size = 640
p = next(model_no.parameters()) # Can get from jit
max_stride = int(model_no.stride.max()) # Must hardcode

def preprocess(img_path):
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    s = img.shape[:2]
    g = (size / max(s))
    shape1 = [[y * g for y in s]]
    shape1 = [make_divisible(x, max_stride) for x in np.stack(shape1, 0).max(0)]  # inference shape
    x = letterbox(img1, new_shape=shape1, auto=False)[0]   # pad
    x = np.ascontiguousarray(x[None].transpose((0, 3, 1, 2)))  # BHWC to BCHW
    img_tensor = torch.from_numpy(x).to(p.device).type_as(p) / 255.  # uint8 to fp16/32
    return img, img_tensor, shape1

In [52]:
conf = 0.25
iou = 0.45
classes = None
max_det = 1000

def postprocess(img, shape1, output, img_tensor):
    
    # Inference
    y = output[0]  # forward

    # Post-process
    y = non_max_suppression(y, conf, iou_thres=iou, classes=classes, max_det=max_det)  # NMS

    scale_coords(shape1, y[0][:, :4], img.shape[:2])
    
    return Detections(imgs=[img], pred=y, files=[], times=[1,2,3,4], names=model_no.names, shape=img_tensor.shape
        )

In [53]:
img_original, img_tensor, shape1 = preprocess(img1_path)
output = model_no(img_tensor)
detections = postprocess(img_original, shape1, output, img_tensor)
result_3 = detections.pandas().xyxy[0]
result_1.equals(result_3)

True

## Saving as TorchScript

In [86]:
model_path = "../street2sat_utils/model_weights/best.torchscript.pt"
ts = torch.jit.trace(model_no, img_tensor, strict=False)
ts.save(model_path)

/Users/izvonkov/nasaharvest/street2sat_website/venv/lib/python3.7/site-packages/yolov5/models/yolo.py:55: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4] or self.onnx_dynamic:


In [87]:
model_jit = torch.jit.load(model_path)

In [91]:
output_jit = model_jit(img_tensor)
detections = postprocess(img1, shape1, output_jit, img_tensor)
result_4 = detections.pandas().xyxy[0]

In [92]:
result_1.equals(result_4)

True

## Comparing to:
https://github.com/louisoutin/yolov5_torchserve/blob/master/ressources/torchserve_handler.py

In [117]:
model_no.stride.max()

tensor(32.)